In [ ]:
credibility,
balance, 
duration,
history,
purpose,
credit_amount,
savings,
employment,
instPercent,
sexMarried,
guarantors,
residence,
assets,
age,
conc_credit,
housing,
credits,
job,
dependents,
has_phone,
foreign

In [40]:
import pandas as pd
PATH = "/home/biennt/Documents/Source_code/sparkProject/codeSpark"
data = sc.textFile(PATH+"/data/german.data")
data1 = pd.read_csv(PATH+"/data/germancredit.csv",header=None, names=cols)


In [61]:


splited = data.flatMap(lambda file: file.split("\n")) \
                .map(lambda lines: lines.split(" "))
#                 .map(lambda lines: [float(i) for i in lines]) \
#                 .map(lambda lines: [lines[0], lines[1]-1, lines[2], \
#                                     lines[3], lines[4], lines[5], \
#                                     lines[6]-1, lines[7]-1, lines[8], \
#                                     lines[9]-1, lines[10]-1, lines[11]-1, \
#                                     lines[12]-1, lines[13], lines[14]-1, \
#                                     lines[15]-1,lines[16]-1, lines[17]-1, \
#                                     lines[18]-1, lines[19]-1, lines[20]-1 \
#                                    ])
                
# splited.collect()

In [35]:
cols = ['balance', 
        'duration',
        'history',
        'purpose',
        'credit_amount',
        'savings',
        'employment',
        'instPercent',
        'sexMarried',
        'guarantors',
        'residence',
        'assets',
        'age',
        'conc_credit',
        'housing',
        'credits',
        'job',
        'dependents',
        'hasPhone',
        'foreign',
        'credibility'
        ]

In [36]:
df = splited.toDF(cols)
display(df)

DataFrame[balance: string, duration: string, history: string, purpose: string, credit_amount: string, savings: string, employment: string, instPercent: string, sexMarried: string, guarantors: string, residence: string, assets: string, age: string, conc_credit: string, housing: string, credits: string, job: string, dependents: string, hasPhone: string, foreign: string, credibility: string]

In [89]:
from pyspark.ml.feature import StringIndexer
INDEXED = '_indexed'

for idx, val in enumerate(cols):
    if idx in [1,4,7,10,12,15,17]:
        df = df.withColumn(val, df[val].cast(FloatType()))

balance = StringIndexer(inputCol=cols[0], outputCol=cols[0]+ INDEXED)

history = StringIndexer(inputCol=cols[2], outputCol=cols[2]+ INDEXED)

purpose = StringIndexer(inputCol=cols[3], outputCol=cols[3]+ INDEXED)

savings = StringIndexer(inputCol=cols[5], outputCol=cols[5]+ INDEXED)

employment = StringIndexer(inputCol=cols[6], outputCol=cols[6]+ INDEXED)

sexMarried = StringIndexer(inputCol=cols[8], outputCol=cols[8]+ INDEXED)

guarantors = StringIndexer(inputCol=cols[9], outputCol=cols[9]+ INDEXED)

assets = StringIndexer(inputCol=cols[11], outputCol=cols[11]+ INDEXED)

conc_credit = StringIndexer(inputCol=cols[13], outputCol=cols[13]+ INDEXED)

housing = StringIndexer(inputCol=cols[14], outputCol=cols[14]+ INDEXED)

job = StringIndexer(inputCol=cols[16], outputCol=cols[16]+ INDEXED)

hasPhone = StringIndexer(inputCol=cols[18], outputCol=cols[18]+ INDEXED)

foreign = StringIndexer(inputCol=cols[19], outputCol=cols[19]+ INDEXED)

credibility = StringIndexer(inputCol=cols[20], outputCol=cols[20]+ INDEXED)



In [90]:
indexedCols = ['balance_indexed',
               'duration',
               'history_indexed',
               'purpose_indexed',
               'credit_amount',
               'savings_indexed',
               'employment_indexed',
               'instPercent',
               'sexMarried_indexed',
               'guarantors_indexed',
               'residence',
               'assets_indexed',
               'age',
               'conc_credit_indexed',
               'housing_indexed',
               'credits',
               'job_indexed',
               'dependents',
               'hasPhone_indexed',
               'foreign_indexed',
               'credibility_indexed'
]

In [91]:
balance_model = balance.fit(df).transform(df)
history_model = history.fit(df).transform(balance_model)
purpose_model = purpose.fit(df).transform(history_model)
savings_model = savings.fit(df).transform(purpose_model)
employment_model = employment.fit(df).transform(savings_model)
sexMarried_model = sexMarried.fit(df).transform(employment_model)
guarantors_model = guarantors.fit(df).transform(sexMarried_model)
assets_model = assets.fit(df).transform(guarantors_model)
conc_credit_model = conc_credit.fit(df).transform(assets_model)
housing_model = housing.fit(df).transform(conc_credit_model)
job_model = job.fit(df).transform(housing_model)
hasPhone_model = hasPhone.fit(df).transform(job_model)
foreign_model = foreign.fit(df).transform(hasPhone_model)
credibility_model = credibility.fit(df).transform(foreign_model)
data_prepared = credibility_model.select(indexedCols)

data_prepared.show(5)

+---------------+--------+---------------+---------------+-------------+---------------+------------------+-----------+------------------+------------------+---------+--------------+----+-------------------+---------------+-------+-----------+----------+----------------+---------------+-------------------+
|balance_indexed|duration|history_indexed|purpose_indexed|credit_amount|savings_indexed|employment_indexed|instPercent|sexMarried_indexed|guarantors_indexed|residence|assets_indexed| age|conc_credit_indexed|housing_indexed|credits|job_indexed|dependents|hasPhone_indexed|foreign_indexed|credibility_indexed|
+---------------+--------+---------------+---------------+-------------+---------------+------------------+-----------+------------------+------------------+---------+--------------+----+-------------------+---------------+-------+-----------+----------+----------------+---------------+-------------------+
|            1.0|     6.0|            1.0|            0.0|       1169.0|    

In [92]:
featureCols = ['balance_indexed',
               'duration',
               'history_indexed',
               'purpose_indexed',
               'credit_amount',
               'savings_indexed',
               'employment_indexed',
               'instPercent',
               'sexMarried_indexed',
               'guarantors_indexed',
               'residence',
               'assets_indexed',
               'age',
               'conc_credit_indexed',
               'housing_indexed',
               'credits',
               'job_indexed',
               'dependents',
               'hasPhone_indexed',
               'foreign_indexed'
]

In [93]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

assembler = VectorAssembler(inputCols=featureCols, outputCol="indexedFeatures")
df2 = assembler.transform(data_prepared)
labelIndexer = StringIndexer(inputCol="credibility_indexed", outputCol="indexedLabel")
data = labelIndexer.fit(df2).transform(df2)

data.show()

+---------------+--------+---------------+---------------+-------------+---------------+------------------+-----------+------------------+------------------+---------+--------------+----+-------------------+---------------+-------+-----------+----------+----------------+---------------+-------------------+--------------------+------------+
|balance_indexed|duration|history_indexed|purpose_indexed|credit_amount|savings_indexed|employment_indexed|instPercent|sexMarried_indexed|guarantors_indexed|residence|assets_indexed| age|conc_credit_indexed|housing_indexed|credits|job_indexed|dependents|hasPhone_indexed|foreign_indexed|credibility_indexed|     indexedFeatures|indexedLabel|
+---------------+--------+---------------+---------------+-------------+---------------+------------------+-----------+------------------+------------------+---------+--------------+----+-------------------+---------------+-------+-----------+----------+----------------+---------------+-------------------+---------

In [94]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [95]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")


In [96]:
model = dt.fit(trainingData)
predictions = model.transform(testData)


predictions.select(["prediction", "indexedLabel","indexedFeatures"]).show(5)



+----------+------------+--------------------+
|prediction|indexedLabel|     indexedFeatures|
+----------+------------+--------------------+
|       0.0|         0.0|(20,[1,2,4,6,7,10...|
|       0.0|         0.0|(20,[1,3,4,6,7,10...|
|       0.0|         0.0|(20,[1,3,4,7,9,10...|
|       0.0|         0.0|(20,[1,3,4,7,8,10...|
|       0.0|         0.0|[0.0,6.0,0.0,9.0,...|
+----------+------------+--------------------+
only showing top 5 rows



In [97]:
print(model.toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_ab21008e300f) of depth 5 with 43 nodes
  If (feature 0 in {0.0,3.0})
   If (feature 13 in {1.0,2.0})
    If (feature 3 in {0.0,2.0,3.0,5.0})
     If (feature 4 <= 10294.0)
      Predict: 0.0
     Else (feature 4 > 10294.0)
      Predict: 1.0
    Else (feature 3 not in {0.0,2.0,3.0,5.0})
     If (feature 10 <= 1.5)
      Predict: 0.0
     Else (feature 10 > 1.5)
      If (feature 12 <= 44.5)
       Predict: 1.0
      Else (feature 12 > 44.5)
       Predict: 0.0
   Else (feature 13 not in {1.0,2.0})
    If (feature 12 <= 22.5)
     If (feature 5 in {1.0,2.0,3.0})
      Predict: 0.0
     Else (feature 5 not in {1.0,2.0,3.0})
      If (feature 7 <= 2.5)
       Predict: 0.0
      Else (feature 7 > 2.5)
       Predict: 1.0
    Else (feature 12 > 22.5)
     If (feature 4 <= 4126.0)
      Predict: 0.0
     Else (feature 4 > 4126.0)
      If (feature 6 in {0.0,1.0,2.0,4.0})
       Predict: 0.0
      Else (feature 6 not in {0.0,1.0,2.0,

In [98]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))



Test Error = 0.254019 
